In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 75531757,
 'calculationPrice': 'close',
 'change': -5.24,
 'changePercent': -0.03278,
 'close': 162.6,
 'closeSource': 'ffiocail',
 'closeTime': 1718341642520,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 156.87,
 'delayedPriceTime': 1691257768754,
 'extendedChange': 0.6,
 'extendedChangePercent': 0.0039,
 'extendedPrice': 160.3,
 'extendedPriceTime': 1657256599551,
 'high': 166.3,
 'highSource': ' eeeyapi drl tucm5n1eid',
 'highTime': 1638435020617,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 162.85,
 'iexCloseTime': 1689252033435,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 163.21,
 'iexOpenTime': 1685654206229,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1708454326244,
 'latestPrice': 156.96,
 'latestSource': 'Close',
 'latestTime': 'November 26, 2021',
 'latestUpdate': 1664428516587,
 'latestVolume': 80325075,
 'low': 161.

In [4]:
pe_ratio = data['peRatio']
pe_ratio

14.02

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,153.66,49.68,N/A
1,AAL,17.96,-3.57,N/A
2,AAP,229.37,23.93,N/A
3,AAPL,164.24,14.12,N/A
4,ABBV,121.81,28.35,N/A
...,...,...,...,...
500,YUM,126.50,24.71,N/A
501,ZBH,125.77,33.19,N/A
502,ZBRA,610.19,39.3,N/A
503,ZION,67.16,6.2,N/A


In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [9]:
portfolio_input()

Enter the value of your portfolio:100000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,775.75,3.5,2578
1,NRG,37.19,3.68,53777
2,EBAY,76.07,4.09,26291
3,MPC,64.26,4.4,31123
4,DHI,99.24,4.43,20153
5,BEN,34.71,4.69,57620
6,TSN,82.55,4.93,24227
7,LYB,90.80,5.38,22026
8,HOLX,77.36,5.59,25853
9,PRU,105.70,5.81,18921


In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.400,N/A,62.31,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,306.080,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,295.110,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,10.07,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.480,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.650,N/A,14.72,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.630,N/A,32.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.850,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2887.660,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,104.960,N/A,34.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.821782
1      0.051485
2      0.493069
3      0.261386
4       0.59802
         ...   
500     0.49802
501    0.667327
502    0.734653
503    0.086139
504    0.855446
Name: PE Percentile, Length: 505, dtype: object
0      0.808911
1      0.055446
2      0.612871
3      0.932673
4       0.90297
         ...   
500    0.053465
501         0.3
502    0.839604
503    0.172277
504    0.942574
Name: PB Percentile, Length: 505, dtype: object
0      0.820792
1      0.047525
2           0.2
3      0.536634
4      0.577228
         ...   
500    0.742574
501    0.533663
502    0.761386
503    0.451485
504    0.930693
Name: PS Percentile, Length: 505, dtype: object
0      0.853465
1      0.021782
2      0.429703
3      0.346535
4      0.316832
         ...   
500    0.691089
501    0.613861
502    0.827723
503    0.069307
504    0.889109
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.843564
1      0.067327
2      0.172277
3      0.623762
4      0.524752
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.10,N/A,47.96,0.821782,9.67,0.808911,7.7000,0.820792,31.250163,0.853465,14.428079,0.843564,N/A
1,AAL,18.04,N/A,-3.70,0.051485,-1.57,0.055446,0.4893,0.047525,-7.618432,0.021782,1.493911,0.067327,N/A
2,AAP,227.58,N/A,23.83,0.493069,4.62,0.612871,1.3500,0.2,12.900505,0.429703,3.064515,0.172277,N/A
3,AAPL,163.51,N/A,13.98,0.261386,20.45,0.932673,3.5200,0.536634,10.935347,0.346535,8.835078,0.623762,N/A
4,ABBV,120.86,N/A,28.77,0.59802,15.90,0.90297,3.8400,0.577228,10.423054,0.316832,7.419892,0.524752,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.90,N/A,24.02,0.49802,-4.70,0.053465,5.6600,0.742574,20.429805,0.691089,9.790677,0.675248,N/A
501,ZBH,127.82,N/A,32.66,0.667327,2.10,0.3,3.4900,0.533663,18.099715,0.613861,5.956702,0.40396,N/A
502,ZBRA,591.27,N/A,38.90,0.734653,11.39,0.839604,5.9400,0.761386,28.619396,0.827723,11.932281,0.764356,N/A
503,ZION,64.90,N/A,6.29,0.086139,1.43,0.172277,2.9000,0.451485,4.905939,0.069307,2.631534,0.128713,N/A


In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.10,N/A,47.96,0.821782,9.67,0.808911,7.7000,0.820792,31.250163,0.853465,14.428079,0.843564,0.829703
1,AAL,18.04,N/A,-3.70,0.051485,-1.57,0.055446,0.4893,0.047525,-7.618432,0.021782,1.493911,0.067327,0.048713
2,AAP,227.58,N/A,23.83,0.493069,4.62,0.612871,1.3500,0.2,12.900505,0.429703,3.064515,0.172277,0.381584
3,AAPL,163.51,N/A,13.98,0.261386,20.45,0.932673,3.5200,0.536634,10.935347,0.346535,8.835078,0.623762,0.540198
4,ABBV,120.86,N/A,28.77,0.59802,15.90,0.90297,3.8400,0.577228,10.423054,0.316832,7.419892,0.524752,0.58396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.90,N/A,24.02,0.49802,-4.70,0.053465,5.6600,0.742574,20.429805,0.691089,9.790677,0.675248,0.532079
501,ZBH,127.82,N/A,32.66,0.667327,2.10,0.3,3.4900,0.533663,18.099715,0.613861,5.956702,0.40396,0.503762
502,ZBRA,591.27,N/A,38.90,0.734653,11.39,0.839604,5.9400,0.761386,28.619396,0.827723,11.932281,0.764356,0.785545
503,ZION,64.90,N/A,6.29,0.086139,1.43,0.172277,2.9000,0.451485,4.905939,0.069307,2.631534,0.128713,0.181584


In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [19]:
portfolio_input()

Enter the value of your portfolio:100000000


In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\Capol\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.34,78926,6.29,0.086139,0.4545,0.057426,0.3779,0.027723,2.958896,0.031683,0.366866,0.007921,0.042178
1,AAL,18.04,110864,-3.70,0.051485,-1.5700,0.055446,0.4893,0.047525,-7.618432,0.021782,1.493911,0.067327,0.048713
2,PRU,106.80,18726,5.92,0.083168,0.6480,0.065347,0.5844,0.051485,4.538773,0.061386,0.589788,0.011881,0.054653
3,BEN,34.76,57537,4.78,0.073267,0.7647,0.077228,1.0374,0.134653,3.076173,0.033663,0.829020,0.019802,0.067723
4,BA,200.16,9992,-14.43,0.041584,-8.3900,0.045545,1.9400,0.311881,-33.738662,0.005941,-1319.515304,0.00198,0.081386
5,DHI,100.66,19868,4.36,0.071287,1.2100,0.125743,0.6443,0.057426,3.395050,0.037624,2.392699,0.116832,0.081782
6,TSN,82.01,24387,4.90,0.075248,0.8716,0.085149,0.3265,0.017822,3.758498,0.041584,3.282303,0.192079,0.082376
7,MET,60.50,33057,10.42,0.172277,0.7608,0.075248,0.7214,0.079208,4.940375,0.071287,0.711672,0.017822,0.083168
8,AIG,58.00,34482,8.87,0.136634,0.7329,0.069307,1.0040,0.126733,4.467390,0.055446,0.995225,0.033663,0.084356
9,MCK,230.15,8689,-7.57,0.047525,-59.6800,0.021782,0.1381,0.007921,8.406955,0.219802,2.929739,0.152475,0.089901


In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [24]:
writer.save()